In [2]:
import io
import itertools

import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [3]:
#loading the data
train_data = np.load(r'datasets/Trousers & Jeans - All - Train.npz')
val_data = np.load(r'datasets/Trousers & Jeans - All - Validation.npz')
test_data = np.load(r'datasets/Trousers & Jeans - All - Test.npz')

In [4]:
train_images, train_labels = train_data['images'], train_data['genders']
val_images, val_labels = val_data['images'], val_data['genders']
test_images, test_labels = test_data['images'], test_data['genders']

In [5]:
# scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [5]:
#constant definition
NUM_EPOCHS = 15
BATCH_SIZE = 64

In [7]:
#hyperparam definition
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([5,7]))
HP_NUM_FILTER = hp.HParam('filters_number', hp.Discrete([64,96,128]))
# HP_FILTER_SIZE_2 = hp.HParam('filter_size_2', hp.Discrete([3,5]))
HP_DENSE_SIZE = hp.HParam('dense_size', hp.Discrete([512,1024]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'D:/sam/Logs/Model 5/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_FILTER_SIZE,HP_NUM_FILTER,HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [8]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],5,activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(4)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'D:/sam/Logs/Model 5/fit/'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=['Male', 'Female'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'D:/sam/saved Models/Model 5/Run-{}'.format(session_num))
    
    return accuracy


In [9]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [ ]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for num_filter in HP_NUM_FILTER.domain.values:
#         for filter_size_2 in HP_FILTER_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
        
                hparams = {
                    HP_FILTER_SIZE : filter_size,
                    HP_NUM_FILTER : num_filter,
                    HP_DENSE_SIZE: dense_size
                }
        
                run_name = 'run-%d' %session_num
                print('---- Starting trial : %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('D:/sam/Logs/Model 5/hparam_tuning/' +run_name, hparams, session_num)
        
                session_num += 1

---- Starting trial : run-1
{'filter_size': 5, 'filters_number': 64, 'dense_size': 512}
Epoch 1/15
63/63 - 68s - loss: 0.8817 - accuracy: 0.6721 - val_loss: 0.4963 - val_accuracy: 0.7640
Epoch 2/15
63/63 - 63s - loss: 0.4401 - accuracy: 0.7922 - val_loss: 0.4449 - val_accuracy: 0.8100
Epoch 3/15
63/63 - 67s - loss: 0.4116 - accuracy: 0.8192 - val_loss: 0.4202 - val_accuracy: 0.8360
Epoch 4/15
63/63 - 62s - loss: 0.3761 - accuracy: 0.8374 - val_loss: 0.4025 - val_accuracy: 0.8240
Epoch 5/15
63/63 - 64s - loss: 0.3412 - accuracy: 0.8609 - val_loss: 0.3873 - val_accuracy: 0.8260
Epoch 6/15
63/63 - 63s - loss: 0.3194 - accuracy: 0.8694 - val_loss: 0.3605 - val_accuracy: 0.8520
Epoch 7/15
63/63 - 66s - loss: 0.2962 - accuracy: 0.8754 - val_loss: 0.3576 - val_accuracy: 0.8280
Epoch 8/15
63/63 - 62s - loss: 0.2951 - accuracy: 0.8784 - val_loss: 0.3710 - val_accuracy: 0.8600
Epoch 9/15
63/63 - 64s - loss: 0.2723 - accuracy: 0.8951 - val_loss: 0.3320 - val_accuracy: 0.8600
Epoch 10/15
63/63 - 6

Epoch 2/15
63/63 - 70s - loss: 0.4564 - accuracy: 0.7947 - val_loss: 0.5299 - val_accuracy: 0.7480
Epoch 3/15
63/63 - 71s - loss: 0.4285 - accuracy: 0.8017 - val_loss: 0.4299 - val_accuracy: 0.8220
Epoch 4/15
63/63 - 70s - loss: 0.3953 - accuracy: 0.8209 - val_loss: 0.4213 - val_accuracy: 0.8340
Epoch 5/15
63/63 - 70s - loss: 0.3541 - accuracy: 0.8482 - val_loss: 0.3672 - val_accuracy: 0.8420
Epoch 6/15
63/63 - 70s - loss: 0.3620 - accuracy: 0.8459 - val_loss: 0.3919 - val_accuracy: 0.8460
Epoch 7/15
63/63 - 70s - loss: 0.3378 - accuracy: 0.8641 - val_loss: 0.4515 - val_accuracy: 0.7800
16/16 [==============================] - 2s 117ms/step - loss: 0.3672 - accuracy: 0.8420
INFO:tensorflow:Assets written to: D:/sam/saved Models/Model 5/Run-8\assets
---- Starting trial : run-9
{'filter_size': 7, 'filters_number': 96, 'dense_size': 512}
Epoch 1/15
63/63 - 105s - loss: 0.8178 - accuracy: 0.6696 - val_loss: 0.4968 - val_accuracy: 0.7460
Epoch 2/15
63/63 - 104s - loss: 0.4448 - accuracy: 0.

In [6]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"D:/sam/saved Models/Model 5/Run-1")

In [7]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)

16/16 [==============================] - 2s 100ms/step - loss: 0.3024 - accuracy: 0.8880


In [8]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.3024. Test accuracy: 88.80%


In [9]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 5/hparam_tuning"

In [11]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 5/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 14280), started 0:00:09 ago. (Use '!kill 14280' to kill it.)